<a href="https://colab.research.google.com/github/harshari/PlantAI/blob/main/MIDI_to_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
import mido

def midi_to_notes(midi_file, default_tempo=500000):  # Default tempo: 120 bpm
    midi = mido.MidiFile(midi_file)
    notes = []
    tempo = default_tempo  # Microseconds per beat
    for track in midi.tracks:
        current_time = 0  # Reset time for each track
        for msg in track:
            if msg.type == 'set_tempo':
                tempo = msg.tempo
            if msg.type == 'note_on' and msg.velocity > 0:
                note_start_time = current_time
                note = msg.note

                # Accumulate time until the note off event
                for follow_msg in track:
                    if follow_msg.type == 'note_off' or (follow_msg.type == 'note_on' and follow_msg.velocity == 0 and follow_msg.note == note):
                        note_end_time = note_start_time + follow_msg.time
                        duration = mido.tick2second(note_end_time - note_start_time, midi.ticks_per_beat, tempo)
                        notes.append((note, msg.velocity, round(duration * 1000)))  # Duration in milliseconds
                        break
                    note_start_time += follow_msg.time  # Update time until note off

            # Update current time for the next message
            current_time += msg.time

    return notes

# Example usage
midi_path = '/content/Queen_midi.midi'
notes = midi_to_notes(midi_path)

notes_string = '\n'.join([f'{note},{velocity},{duration}' for note, velocity, duration in notes])

# Write to a text file
with open('queen_notes.txt', 'w') as file:
    file.write(notes_string)

# Print the notes
for note in notes:
    print(note)


# Only issue of the above process is that it loses the sense of chords, as the information about the notes which are played together is lost. How can we capture that??


### A good way might be to also have chords as different MIDI channel

In [11]:
import mido

def midi_to_notes(midi_file, default_tempo=500000, chord_threshold=50):  # Default tempo: 120 bpm
    """Converts a MIDI file to a list of notes or chords with durations.

    Args:
        midi_file: Path to the MIDI file.
        default_tempo: Default tempo in microseconds per beat.
        chord_threshold: Time in milliseconds to consider notes as part of the same chord.

    Returns:
        List of notes or chords, each represented as a string "note1,note2,...,noteN,duration".
    """
    midi = mido.MidiFile(midi_file)
    all_notes = []
    current_time = 0
    note_starts = {}
    tempo = default_tempo

    for track in midi.tracks:
        for msg in track:
            current_time += mido.tick2second(msg.time, midi.ticks_per_beat, tempo) * 1000  # Convert to milliseconds
            if msg.type == 'note_on' and msg.velocity > 0:
                note_starts[msg.note] = current_time
            elif (msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0)) and msg.note in note_starts:
                start_time = note_starts.pop(msg.note)
                duration = round(current_time - start_time)
                all_notes.append((msg.note, duration))

    # Group notes into chords
    chords = []
    while all_notes:
        note, duration = all_notes.pop(0)
        chord = [note]

        for other_note, other_duration in all_notes[:]:
            if abs(duration - other_duration) <= chord_threshold:
                chord.append(other_note)
                all_notes.remove((other_note, other_duration))

        chord_str = ','.join(map(str, chord)) + ',' + str(duration)
        chords.append(chord_str)

    return chords

# Example usage
midi_path = '/content/Queen_midi.midi'
chords = midi_to_notes(midi_path)

chords_string = '\n'.join(chords)

# Write to a text file
with open('midi_chords.txt', 'w') as file:
    file.write(chords_string)

# Print the chords
for chord in chords:
    print(chord)


72,73,73,80,78,81,75,78,79,75,70,71,73,71,88,88,88,88,88,88,87,87,88,88,84,84,78,58,62,65,65,67,72,75,65,67,69,65,64,65,59,57,55,58,45,64,69,62,72,62,72,43,63,34,67,43,63,65,77,52,67,79,85,73,85,73,69,76,81,57,64,61,61,57,64,57,61,64,57,61,64,61,57,64,57,64,61,57,61,64,57,62,66,57,64,61,57,64,61,60,57,63,57,64,61,57,62,66,61,57,64,57,64,61,64,61,57,57,64,61,64,57,61,57,61,64,57,64,61,64,57,61,69,57,69,60,63,60,57,69,63,69,57,64,61,57,62,64,60,67,48,55,60,64,48,67,55,60,64,67,48,55,60,67,55,64,48,47,55,59,67,62,59,67,55,62,58,53,65,46,62,65,46,58,53,62,46,58,65,62,66,65,58,46,66,68,73,65,61,67,64,72,60,64,72,67,60,63,59,71,66,64,72,67,60,68,65,73,61,64,67,60,72,64,72,67,60,60,53,69,65,41,72,65,69,41,53,72,70,74,65,41,53,74,65,41,53,70,71,65,41,53,68,65,41,53,68,71,69,72,41,76,72,69,57,45,65,65,71,55,43,67,67,43,65,71,55,65,67,71,60,67,72,72,74,77,65,68,73,61,65,68,73,61,64,72,67,60,63,59,71,66,63,71,66,64,67,72,60,65,68,61,73,65,73,68,61,67,64,72,60,72,67,60,66,67,64,60,72,69,53,77,72,6